In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("Mi programa")
sc = SparkContext(conf=conf)

In [3]:
import pandas as pd
df = pd.read_csv("C:/Users/Gabo0/Desktop/Jupyter Python/Ejercicios Linkedin/base_datos_2008.csv", nrows = 1000000)

In [4]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,...,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,...,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,...,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,...,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,...,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0


In [5]:
from pyspark.sql.types import StringType
from pyspark import SQLContext
sqlContext = SQLContext(sc)


dfspark = sqlContext.read.format("csv").option("header", "true").option("inferSchema","true").load("C:/Users/Gabo0/Desktop/Jupyter Python/Ejercicios Linkedin/base_datos_2008.csv")

In [8]:
dfspark.show(2)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2008|    1|         3|        4|   2003|      1955|   2211|      2225|           WN|      335

In [9]:
dfspark.head(2)

[Row(Year=2008, Month=1, DayofMonth=3, DayOfWeek=4, DepTime='2003', CRSDepTime=1955, ArrTime='2211', CRSArrTime=2225, UniqueCarrier='WN', FlightNum=335, TailNum='N712SW', ActualElapsedTime='128', CRSElapsedTime='150', AirTime='116', ArrDelay='-14', DepDelay='8', Origin='IAD', Dest='TPA', Distance=810, TaxiIn='4', TaxiOut='8', Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay='NA', WeatherDelay='NA', NASDelay='NA', SecurityDelay='NA', LateAircraftDelay='NA'),
 Row(Year=2008, Month=1, DayofMonth=3, DayOfWeek=4, DepTime='754', CRSDepTime=735, ArrTime='1002', CRSArrTime=1000, UniqueCarrier='WN', FlightNum=3231, TailNum='N772SW', ActualElapsedTime='128', CRSElapsedTime='145', AirTime='113', ArrDelay='2', DepDelay='19', Origin='IAD', Dest='TPA', Distance=810, TaxiIn='5', TaxiOut='10', Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay='NA', WeatherDelay='NA', NASDelay='NA', SecurityDelay='NA', LateAircraftDelay='NA')]

In [10]:
dfspark.count()

7009728

In [11]:
dfspark = dfspark.sample(fraction = 0.001, withReplacement = False)

In [12]:
dfspark.count()

6958

In [13]:
dfspark = dfspark.withColumn("ArrDelay",dfspark["ArrDelay"].cast("integer"))

In [17]:
df2 = dfspark.na.drop(subset = ["ArrDelay", "DepDelay","Distance"])

In [18]:
df2 = df2.filter("ArrDelay is not NULL")

In [19]:
df2.count()

6807

In [20]:
df2.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Ca

In [21]:
import numpy as np
media = np.mean(df2.select("ArrDelay").collect())

In [22]:
media

7.4618774790656675

In [23]:
df2.rdd.getNumPartitions()

6